In [1]:
import threading

def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

def find_primes_in_range(start, end, primes_list):
    for num in range(start, end + 1):
        if is_prime(num):
            primes_list.append(num)

if __name__ == "__main__":
    start_range = 1
    end_range = 1000
    num_threads = 2
    
    primes = []
    
    # Разделяем диапазон на части для каждого потока
    range_per_thread = (end_range - start_range + 1) // num_threads
    
    threads = []
    for i in range(num_threads):
        thread_start = start_range + i * range_per_thread
        thread_end = thread_start + range_per_thread - 1
        if i == num_threads - 1: # Убедимся, что последний поток охватывает оставшиеся числа
            thread_end = end_range
            
        thread = threading.Thread(target=find_primes_in_range, args=(thread_start, thread_end, primes))
        threads.append(thread)
        thread.start()
    
    # Ждем завершения всех потоков
    for thread in threads:
        thread.join()
        
    primes.sort() # Сортируем найденные простые числа
    print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]


In [2]:
import threading
from collections import defaultdict

def count_words_in_chunk(chunk, word_counts):
    words = chunk.lower().split()
    for word in words:
        # Убираем знаки препинания
        cleaned_word = ''.join(char for char in word if char.isalnum())
        if cleaned_word:
            word_counts[cleaned_word] += 1

def read_file_in_chunks(filepath, chunk_size):
    with open(filepath, 'r', encoding='utf-8') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            yield chunk

if __name__ == "__main__":
    file_path = "large_text_file.txt"  # Укажи путь к своему файлу
    num_threads = 4  # Можешь изменить количество потоков
    chunk_size = 4096  # Размер чанка для чтения файла (например, 4KB)

    # Создадим фиктивный большой файл для демонстрации
    with open(file_path, 'w', encoding='utf-8') as f:
        for _ in range(10000):
            f.write("это пример большого текстового файла для подсчета слов. слова повторяются. пример слова.\n")

    global_word_counts = defaultdict(int)
    
    # Lock для безопасного обновления общего словаря
    lock = threading.Lock()

    def thread_task(chunk, local_word_counts):
        count_words_in_chunk(chunk, local_word_counts)
        with lock:
            for word, count in local_word_counts.items():
                global_word_counts[word] += count

    threads = []
    local_word_counts_list = []

    for chunk in read_file_in_chunks(file_path, chunk_size):
        local_counts = defaultdict(int)
        local_word_counts_list.append(local_counts)
        thread = threading.Thread(target=thread_task, args=(chunk, local_counts))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    # Выводим отсортированные результаты
    sorted_word_counts = sorted(global_word_counts.items(), key=lambda item: item[1], reverse=True)
    
    print("Отчет по количеству слов:")
    for word, count in sorted_word_counts:
        print(f"'{word}': {count}")

Отчет по количеству слов:
'слова': 19982
'пример': 19976
'слов': 9998
'это': 9995
'для': 9995
'файла': 9990
'большого': 9982
'подсчета': 9982
'текстового': 9978
'повторяются': 9975
'п': 10
'а': 10
'о': 8
'сл': 8
'с': 6
'сло': 6
'пр': 5
'имер': 5
'прим': 5
'ер': 5
'ого': 5
'го': 5
'ова': 5
'по': 5
'я': 5
'ример': 5
'при': 5
'мер': 5
'приме': 4
'р': 4
'лова': 4
'ва': 4
'эт': 3
'б': 3
'ольшого': 3
'бол': 3
'ьшого': 3
'больш': 3
'большог': 3
'те': 3
'кстового': 3
'текс': 3
'тового': 3
'тексто': 3
'вого': 3
'текстово': 3
'ф': 3
'айла': 3
'фай': 3
'ла': 3
'д': 3
'ля': 3
'одсчета': 3
'под': 3
'счета': 3
'подсч': 3
'ета': 3
'подсчет': 3
'ов': 3
'вторяются': 3
'повт': 3
'оряются': 3
'повтор': 3
'яются': 3
'повторяю': 3
'тся': 3
'повторяютс': 3
'э': 2
'то': 2
'бо': 2
'льшого': 2
'боль': 2
'шого': 2
'большо': 2
'т': 2
'екстового': 2
'тек': 2
'стового': 2
'текст': 2
'ового': 2
'текстов': 2
'текстовог': 2
'фа': 2
'йла': 2
'файл': 2
'дл': 2
'дсчета': 2
'подс': 2
'чета': 2
'подсче': 2
'та': 2
'лов': 